In [ ]:
"""
評価指標: roc_auc
最終締切: 2021年7月28日

信号を注入した後、各スニペットを正規化するので、対応する配列の過剰なエネルギーを探すだけでは、ほとんどの針を識別することはできません。
そのため、オンターゲットの観測データのみに現れるパターンを探すような、より繊細なアルゴリズムが必要になるでしょう。

すべての「針」信号が斜線のように見えるわけではありませんし、
3つの「A」観測のすべてに存在するわけではありませんが、共通しているのは、「A」観測の一部またはすべてにしか存在しないということです（ケイデンススニペットのパネル1、3、5）。
あなたの課題は、干し草の山からの誤検出を最小限に抑えながら、できるだけ多くの針を見つけるアルゴリズムを訓練することです。
"""

In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import os

In [2]:
df = pd.read_csv("../train_labels.csv")
df.shape

(60000, 2)

In [3]:
df.head()

,id,target
0,0000799a2b2c42d,0
1,00042890562ff68,0
2,0005364cdcb8e5b,0
3,0007a5a46901c56,0
4,0009283e145448e,0


In [4]:
# 1 file 1 target
df["target"].value_counts()

0    54000
1     6000
Name: target, dtype: int64

In [13]:
# augmentation test
import albumentations as albu
from albumentations.pytorch import ToTensorV2

def get_augmentation(mode="train"):
    if mode == "train":
        transform = [
            ToTensorV2(),
        ]
    else:
        transform = [
            ToTensorV2(),
        ]
    return albu.Compose(transform)

def data_augmentation(transform, image):
    augmented = transform(image=image)
    image = augmented['image']
    return image

transform = get_augmentation("train")
test = data_augmentation(transform, image)
test.shape

torch.Size([1, 256, 1638])

In [5]:
train_df = pd.read_csv("../train_labels.csv")
test_df = pd.read_csv("../sample_submission.csv")

def get_train_file_path(image_id):
    return "train/{}/{}.npy".format(image_id[0], image_id)

def get_test_file_path(image_id):
    return "test/{}/{}.npy".format(image_id[0], image_id)

train_df['file_path'] = train_df['id'].apply(get_train_file_path)
test_df['file_path'] = test_df['id'].apply(get_test_file_path)

In [6]:
# CV Split
from sklearn.model_selection import StratifiedKFold

n_fold = 5
kf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(kf.split(train_df, train_df["target"])):
    train_df.loc[val_index, 'fold'] = int(n)
train_df['fold'] = train_df['fold'].astype(int)
train_df.groupby(['fold', 'target']).size()

fold  target
0     0         10800
      1          1200
1     0         10800
      1          1200
2     0         10800
      1          1200
3     0         10800
      1          1200
4     0         10800
      1          1200
dtype: int64

In [7]:
train_df.head()

,id,target,file_path,fold
0,0000799a2b2c42d,0,train/0/0000799a2b2c42d.npy,2
1,00042890562ff68,0,train/0/00042890562ff68.npy,3
2,0005364cdcb8e5b,0,train/0/0005364cdcb8e5b.npy,3
3,0007a5a46901c56,0,train/0/0007a5a46901c56.npy,2
4,0009283e145448e,0,train/0/0009283e145448e.npy,2


In [8]:
train_df.to_csv("../train_labels.csv", index=False)

In [10]:
test_df.to_csv("../sample_submission.csv", index=False)